# Summary

- Initial exploratory data analysis of complaint file


### Notes
- Data dictionary is not helpful.
- We have no firmographic data about the companies.
- How do we want to deal with fake reviews?
- Month, permo and predicted value have 100% null values.
- Recommend business has 33% null.
- Possible issues with duplicate records.
    - EDA reveals that ~ 67% of records are duplicates. Are we working with the entire negative datast?
    - Business ID: 27139514, Charles Sneed, submitted the sample complaint 4,000 times.
    - The response from the business appears to be exactly the same.
    - Description Field: complaint exactly the same yet description changes.
    - Complaints all submitted on the same date.
- Recommend business: how do should we interpret a positive response to this despite a complaint?

- NER
    - I wonder if we can use the LLM to extract amounts mentioned in the text, for example, amounts paid, lost, damage.
    - This could be an indication of the magnitude of the claim for a given company (not across companies).
- Summarization
    - Ask LLM to summarize the complaint.  What is the root of the complaint?  This may help human get to conclusion
      faster.  Example, root of complaint.
    - Ask LLM to predict what they think the user wants, i.e. do they want a refund?  Do they want to go to court?
    - Do they simply want the company to respond to them?
    - Does the customer mention the first date on which the issue started?
    - Does the recommendation to continue with the provider match the review?
- Status
    - Unclear if this field is useful.  99% are reviewed and published.


### Transformations to Data
- Data type conversion
- Drop duplicates.

### Data Dictionary
- Description: Is this a classification of the complaint or associated with the provider?  Who fills out this field?
- isPrimary: not included in the data dictionary.
- Business: name of business
- First Name: first name of person submitting review.
- Last Name: last name of person submitting review. 
- Submitted: date submitted.
- Source: has 5 levels.  'IABBB/Profile', nan, 'Review Partner', 'Transfer', 'blue Form'.  What do they mean?
- Experience: indicates the polarity of the review.  In this dataset we only have negative.
- Recommend Business:
- Status: indicates the status of the complaint.  99% are reviewed and published.
- Code: unclear.  600 unique values.  Not a Pkey.

<br>


---
# I. Setup
---

### Import Libraries

In [1]:
import os
import pandas as pd

### Globals

In [9]:
DIR_ROOT = "/Users/temp-admin/repositories/customer-service-llm"
DIR_DATA = os.path.join(DIR_ROOT, 'data')
DIR_DATA_EDA = os.path.join(DIR_DATA, 'eda')
DIR_DATA_SRC = os.path.join(DIR_DATA, 'source')

FILE_NAME_NEG_REVIEWS = "business_responses_neg.csv"
FILE_NAME_ALL_REVIEWS = "business_responses_all.csv"
FILE_NAME_DD = "data_dictionary.xlsx"

#### Settings

In [25]:
pd.set_option('display.max_columns', None)

### Schema

In [4]:
schema = {
    'numeric_columns': [
        'Business ID', 'PKID', 'FScore', 'Number of Business Responses',
        'Stars', 'Month', 'permno', 'PredictedValue', 'year', 'BID',
        'isPrimary'
    ],
    'categorical_columns': [
        'Business', 'Last Name', 'First Name', 'Email', 'Display Name', 'Zip Code', 'IP', 'Source',
        'Submitted', 'Experience', 'Recommend Business', 'Last Business Response', 'Review',
        'Business Response', 'Status', 'Reason', 'Code', 'Description'
    ]
}

### Load Data

In [21]:
all_df = pd.read_csv(os.path.join(DIR_DATA_SRC, FILE_NAME_ALL_REVIEWS))
all_df.shape

/var/folders/z5/785ggch143j6xcyg8q23rgsw0000gp/T/ipykernel_70296/1135274062.py:1: DtypeWarning: Columns (5,12) have mixed types. Specify dtype option on import or set low_memory=False.
  all_df = pd.read_csv(os.path.join(DIR_DATA_SRC, FILE_NAME_ALL_REVIEWS))


(105619, 25)

In [15]:
neg_df = pd.read_csv(os.path.join(DIR_DATA_SRC, FILE_NAME_NEG_REVIEWS))
neg_df.shape

(26423, 29)

In [6]:
dd_df = pd.read_excel(os.path.join(DIR_DATA_SRC, FILE_NAME_DD))
dd_df.shape

(19, 2)

---
# II. EDA
---

### Display Data

In [28]:
all_df.head(1).transpose()

,0
BBB,Atlanta
CID,115912.0
Consumer Name,Shirley R. F.
Consumer Postal Code,35466
Consumer Email,shisley4@yahoo.com
Consumer Phone,2052922639
Consumer IP Address,64.234.50.187
BID,28121357
Business Name,SoGoodBB
Business Street,2605 N Berkeley Lake Rd NW STE 600


In [29]:
neg_df.head(1).transpose()

,0
Business ID,595.0
Business,"Financial Asset Management Systems, Inc."
PKID,203318.0
Last Name,Doolin
First Name,Mary
Email,stetsoncowgirl@gmail.com
Display Name,Mary D
Zip Code,32504
IP,69.85.205.227
FScore,0.0


In [8]:
dd_df.head()

,Business iD,Unnamed: 1
0,Business,NaN
1,PKID,NaN
2,Last Name,NaN
3,First Name,NaN
4,Email,NaN


### Check Column Diff

In [24]:
all_df.columns

Index(['BBB', 'CID', 'Consumer Name', 'Consumer Postal Code', 'Consumer Email',
       'Consumer Phone', 'Consumer IP Address', 'BID', 'Business Name',
       'Business Street', 'Business City', 'Business State',
       'Business Postal Code', 'Business TOB', 'Tier', 'AB', 'Rtg', 'Received',
       'Stars', 'Verified', 'Published', 'F Score', 'Text', 'Unnamed: 23',
       'Unnamed: 0'],
      dtype='object')

In [19]:
neg_df.columns

Index(['Business ID', 'Business', 'PKID', 'Last Name', 'First Name', 'Email',
       'Display Name', 'Zip Code', 'IP', 'FScore', 'Source', 'Submitted',
       'Experience', 'Recommend Business', 'Number of Business Responses',
       'Last Business Response', 'Review', 'Business Response', 'Status',
       'Reason', 'Stars', 'Month', 'permno', 'PredictedValue', 'year', 'BID',
       'Code', 'isPrimary', 'Description'],
      dtype='object')

### Structural Analysis

#### Describe Data

In [30]:
all_df.describe().columns

Index(['CID', 'BID', 'Stars', 'F Score', 'Unnamed: 23', 'Unnamed: 0'], dtype='object')

#### Get Null Pct By Column

In [32]:
all_df.isna().sum() / all_df.shape[0] * 100

BBB                       0.000000
CID                       8.613980
Consumer Name             8.613980
Consumer Postal Code      9.539950
Consumer Email            8.613980
Consumer Phone           58.061523
Consumer IP Address       8.657533
BID                       0.000000
Business Name             8.613980
Business Street           8.999328
Business City             8.619661
Business State            8.620608
Business Postal Code      8.623448
Business TOB              8.613980
Tier                      8.613980
AB                        8.613980
Rtg                       8.613980
Received                  8.613980
Stars                     8.613980
Verified                  8.613980
Published                 8.613980
F Score                   8.613980
Text                      8.614927
Unnamed: 23             100.000000
Unnamed: 0               91.386020
dtype: float64

In [34]:
all_df["Unnamed: 0"].unique()

array([      nan, 0.000e+00, 1.000e+00, ..., 9.095e+03, 9.096e+03,
       9.097e+03])

---
# Inspect Categorical Columns
---

### i. Get Categorical Column Levels

In [11]:
cat_levels = {}
cat_level_cnts = {}

In [12]:
for c in schema['categorical_columns']:
    lvls = list(set(review_df[c].values.tolist()))
    cat_levels[c] = lvls
    cat_level_cnts[c] = len(lvls)

In [13]:
cat_level_cnts

{'Business': 342,
 'Last Name': 4203,
 'First Name': 2730,
 'Email': 5782,
 'Display Name': 5055,
 'Zip Code': 4243,
 'IP': 5778,
 'Source': 5,
 'Submitted': 697,
 'Experience': 1,
 'Recommend Business': 3,
 'Last Business Response': 569,
 'Review': 5786,
 'Business Response': 4709,
 'Status': 9,
 'Reason': 6,
 'Code': 646,
 'Description': 643}

In [14]:
review_df['Source'].head()

0    IABBB/Profile
1    IABBB/Profile
2    IABBB/Profile
3    IABBB/Profile
4    IABBB/Profile
Name: Source, dtype: object

In [15]:
cat_levels['Experience']

['Negative']

### ii. Inspect Review

In [16]:
customer_reviews = (
    review_df
    .groupby("Review")['Review']
    .count()
    .sort_values(ascending=False)
)
cr_df = pd.DataFrame({
    "reviews": customer_reviews.index.tolist(),
    "count": customer_reviews.values
    }
)

In [17]:
cr_df.head()

,reviews,count
0,"I've spent close to $16,000 with all good Plum...",74
1,Terrible customer service and price gouging.\n...,74
2,My Rinnai tankless water heater stopped heatin...,66
3,I wanted to purchase a home ac unit and since ...,66
4,"I bought a unit at the end of December, spendi...",66


In [18]:
# Subset Reviews to Most Frequent Complaint
top_complaint = review_df[review_df['Review'] == cr_df.iloc[0, 0]]

In [19]:
top_complaint.head(3)

,Business ID,Business,PKID,Last Name,First Name,Email,Display Name,Zip Code,IP,FScore,Source,Submitted,Experience,Recommend Business,Number of Business Responses,Last Business Response,Review,Business Response,Status,Reason,Stars,Month,permno,PredictedValue,year,BID,Code,isPrimary,Description
23238,27139514.0,"All Good Plumbing, Electric, Heating & Cooling",218742.0,Sneed,Charles,myintrinsicideas4u@gmail.com,Charles S,30349,172.58.7.110,0.0,IABBB/Profile,02/17/2023,Negative,True,1.0,05/11/2023,"I've spent close to $16,000 with all good Plum...","Charles, thank you for reaching out to share y...",Reviewed and Published,NaN,1.0,NaN,NaN,NaN,2023,27139514.0,10006-100,0.0,Kitchen Remodel
23239,27139514.0,"All Good Plumbing, Electric, Heating & Cooling",218742.0,Sneed,Charles,myintrinsicideas4u@gmail.com,Charles S,30349,172.58.7.110,0.0,IABBB/Profile,02/17/2023,Negative,True,1.0,05/11/2023,"I've spent close to $16,000 with all good Plum...","Charles, thank you for reaching out to share y...",Reviewed and Published,NaN,1.0,NaN,NaN,NaN,2023,27139514.0,10177-101,0.0,Bathroom Remodel
23240,27139514.0,"All Good Plumbing, Electric, Heating & Cooling",218742.0,Sneed,Charles,myintrinsicideas4u@gmail.com,Charles S,30349,172.58.7.110,0.0,IABBB/Profile,02/17/2023,Negative,True,1.0,05/11/2023,"I've spent close to $16,000 with all good Plum...","Charles, thank you for reaching out to share y...",Reviewed and Published,NaN,1.0,NaN,NaN,NaN,2023,27139514.0,10003-000,0.0,Air Conditioning Contractor


In [20]:
# top_complaint.to_csv(os.path.join(DIR_DATA_EDA, "top_comp laint.csv"))

### iii. inspect status field

In [21]:
cat_levels['Status']

['Rejected',
 'Not a Customer',
 'Not a Customer (Employee)',
 'Duplicate Review',
 'Customer Submitted Complaint',
 'Pending (Under Review)',
 'Not a Customer (Business)',
 'Retracted by Consumer',
 'Reviewed and Published']

In [22]:
review_df.groupby('Status')['Status'].count()

Status
Customer Submitted Complaint        5
Duplicate Review                   11
Not a Customer                     16
Not a Customer (Business)           1
Not a Customer (Employee)           1
Pending (Under Review)              9
Rejected                            4
Retracted by Consumer               9
Reviewed and Published          26367
Name: Status, dtype: int64

### iv. Drop Duplicates

In [36]:
deduped_df = review_df[[c for c in review_df.columns if c not in ['Description', 'Code']]]

In [37]:
deduped_df.shape

(26423, 27)

In [40]:
deduped_df.head(1)

,Business ID,Business,PKID,Last Name,First Name,Email,Display Name,Zip Code,IP,FScore,Source,Submitted,Experience,Recommend Business,Number of Business Responses,Last Business Response,Review,Business Response,Status,Reason,Stars,Month,permno,PredictedValue,year,BID,isPrimary
0,595.0,"Financial Asset Management Systems, Inc.",203318.0,Doolin,Mary,stetsoncowgirl@gmail.com,Mary D,32504,69.85.205.227,0.0,IABBB/Profile,11/08/2022,Negative,True,1.0,11/14/2022,Why are you sending me letters from a loan tha...,FAMS is not a debt buyer and did not purchase ...,Reviewed and Published,NaN,1.0,NaN,NaN,NaN,2022,595.0,0.0


In [46]:
print(f"Pct Duplicates => {1 - (deduped_df.drop_duplicates().shape[0] / review_df.shape[0])}")

Pct Duplicates => 0.6113991598228816
